In [47]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Dec 29 05:05:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |  16057MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.36.0
Uninstalling transformers-4.36.0:
  Successfully uninstalled transformers-4.36.0
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 6.8 MB/s eta 0:00:00
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/f7/fc/c55e5a2da345c9a24aa2e1e0f60eb2ca290b6a41be82da03a6d4baec4f99/accelerate-0.25.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 76.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 27.1 MB/s eta 0:00:00


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import torch

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [8]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [9]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [10]:
dataset_pubmed = load_dataset("ccdv/pubmed-summarization")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset pub_med_summarization_dataset downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___pub_med_summarization_dataset/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
dataset_pubmed

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})

In [12]:
dataset_pubmed['train'] = dataset_pubmed['train'][:5000]

In [13]:
dataset_pubmed["train"]["article"][1]

'it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy . \n preliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer . \n thus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management . \n blood tra

In [14]:
dataset_pubmed["train"]["abstract"][1]

'backgroundanemia in patients with cancer who are undergoing active therapy is commonly encountered and may worsen quality of life in these patients . the effect of blood transfusion is often temporary and may be associated with serious adverse events . \n erythropoiesis - stimulating agents are not effective in 30%50% of patients and may have a negative effect on overall survival.aimsto assess the efficacy and feasibility of intravenous iron therapy in patients with cancer who have non - iron - deficiency anemia and who are undergoing treatment with chemotherapy without the use of erythropoiesis - stimulating agents.methodsadult patients with solid cancers and non - iron - deficiency anemia were included . \n ferric sucrose at a dose of 200 mg was given in short intravenous infusions weekly for a total of 12 weeks . \n hemoglobin level was measured at baseline , every 3 weeks , and 2 weeks after the last iron infusion ( week 14 ) . \n adverse events related to intravenous iron were pr

In [15]:
split_lengths = [len(dataset_pubmed[split])for split in dataset_pubmed]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_pubmed['train'].keys()}")
print("\nArticle:")

print(dataset_pubmed["test"][1]["article"])

print("\nAbstract:")

print(dataset_pubmed["test"][1]["abstract"])

Split lengths: [2, 6633, 6658]
Features: dict_keys(['article', 'abstract'])

Article:
small non - coding rnas are transcribed into mrna but remain untranslated in eukaryotic cells . 
 they include sirna ( small interfering rna ) , mirna ( microrna ) , pirna ( piwi - interacting rna ) and snorna ( small nucleolar rna ) . 
 mirnas are a class of multifunctional singled - stranded small rna which are ~20  nt in length and regulate the stability or translational efficiency of targeted messenger rna depending on the base - pairing complementarity between the mirna and its target mrna [ 1 , 2 ] . although over 1,000 mirna sequences have been identified from the tissues or cells of human origin and other species , as many as 1,000 to 10,000 mirnas per genome have been predicted [ 3 , 4 ] . 
 mirnas regulate a broad range of biological processes including timing of development , cell cycle progression , stem cell self - renewal , differentiation , cancer initiation , cancer cell proliferation 

In [16]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['article'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['abstract'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [17]:
from datasets import Dataset

In [18]:
for split in dataset_pubmed:
     dataset_pubmed_pt = dataset_pubmed[split]  # Access the split

     # Only convert to Dataset if it's not already one
     if not isinstance(dataset_pubmed_pt, Dataset):
        dataset_pubmed_pt = Dataset.from_dict(dataset_pubmed_pt)

     dataset_pubmed_pt = dataset_pubmed_pt.map(convert_examples_to_features, batched=True)  # Apply map


  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [19]:
dataset_pubmed_pt

Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6658
})

In [20]:
dataset_pubmed_pt["attention_mask"][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [21]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [22]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-pubmed', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [23]:
dataset_pubmed_pt.column_names

['article', 'abstract', 'input_ids', 'attention_mask', 'labels']

In [24]:
from datasets import DatasetDict

In [25]:
from datasets import Dataset

# Assuming the entire dataset is currently in `dataset_pubmed_pt`
train_split = dataset_pubmed_pt.train_test_split(test_size=0.2)  # Split into 80% train, 20% test
dataset_pubmed_pt = DatasetDict({
    'train': train_split['train'],
    'test': train_split['test'],
})


In [26]:
# Assuming the "train" and "test" splits are now available
validation_split = dataset_pubmed_pt['train'].train_test_split(test_size=0.2)  # Split "train" into 80% train, 20% validation
dataset_pubmed_pt = DatasetDict({
    'train': validation_split['train'],
    'test': dataset_pubmed_pt['test'],
    'validation': validation_split['test']  # Assign the newly created validation split
})

In [27]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pubmed_pt["test"],
                  eval_dataset=dataset_pubmed_pt["validation"])

In [28]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=83, training_loss=3.7507367651146577, metrics={'train_runtime': 614.1755, 'train_samples_per_second': 2.169, 'train_steps_per_second': 0.135, 'total_flos': 3779365840674816.0, 'train_loss': 3.7507367651146577, 'epoch': 1.0})

In [29]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="abstract"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [30]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [31]:
score = calculate_metric_on_test_ds(
    dataset_pubmed['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'article', column_summary= 'abstract'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:12<00:00,  2.44s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.016951,0.0,0.015964,0.016795


In [32]:
## Save model
model_pegasus.save_pretrained("pegasus-pubmed-model")

In [33]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [34]:
#Load

tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [35]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_pubmed["test"][0]["article"]
print("Article:")
print(sample_text)


Article:
anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] . 
 although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet , 
 our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . 
 overall , neuropsychiatric symptoms in pd have been shown to be negatively associated with cognitive performance . 
 for example , higher depression scores have been correlated with lower scores on the mini - mental state exam ( mmse ) [ 8 , 9 ] as well as tests of memory and executive functions ( e.g. , attention ) [ 1014 ] . 
 likewise , apathy and anhedonia in pd patients have been associated with executiv

In [36]:
reference = dataset_pubmed["test"][0]["abstract"]

print("\nReference Summary:")
print(reference)




Reference Summary:
research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . 
 previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . 
 this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and neuropsychological assessment . 
 group performance was compared across the following cognitive domains : simple attention / visuomotor processing speed , executive function ( e.g. , set - shifting ) , working memory , language , and memory / new verbal learning . 
 results showed that pda+ performed significantly worse on the digit span forward and backward test and part b of the tr

In [37]:
print(gen_kwargs)

{'length_penalty': 0.8, 'num_beams': 8, 'max_length': 128}


In [46]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your fine-tuned model
model_pegasus_finetuned = AutoModelForSeq2SeqLM.from_pretrained("pegasus-pubmed-model")
tokenizer_finetuned = AutoTokenizer.from_pretrained("tokenizer")

# Tokenize the input sequence
tokenized_input = tokenizer_finetuned(sample_text, max_length=1024, truncation=True, return_tensors="pt")

# Get the input IDs and attention mask
input_ids = tokenized_input["input_ids"]
attention_mask = tokenized_input["attention_mask"]

# Generate summary for each chunk
max_chunk_size = 512  # Choose a suitable chunk size
chunk_start = 0
generated_summaries = []

while chunk_start < len(input_ids[0]):
    # Get the current chunk
    input_chunk = input_ids[:, chunk_start : chunk_start + max_chunk_size]
    attention_mask_chunk = attention_mask[:, chunk_start : chunk_start + max_chunk_size]

    # Generate summary for the current chunk
    summary_chunk = model_pegasus_finetuned.generate(
        input_ids=input_chunk,
        attention_mask=attention_mask_chunk,
        max_length=128,
        length_penalty=0.8,
        num_beams=4,
    )

    # Decode and append the summary
    decoded_summary_chunk = tokenizer_finetuned.decode(summary_chunk[0], skip_special_tokens=True)
    generated_summaries.append(decoded_summary_chunk)

    # Move to the next chunk
    chunk_start += max_chunk_size

# Combine the generated summaries into a single summary
final_summary = " ".join(generated_summaries)
print("Generated Summary:", final_summary)


Generated Summary: anxiety affects quality of life in those living with parkinson's disease ( pd ) more so than overall cognitive status, motor deficits, apathy, and depression. anxiety was significantly correlated with neuropsychological measures of attention and executive and visuospatial functions.<n>anxiety was also significantly correlated with measures of attention and executive and visuospatial functions.
